Imports

In [1]:
# Imports
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from pprint import pprint
from nsepython import *
from datetime import datetime
import time

import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

import streamlit as st

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\singh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Get Company Tickers from [NSE Website](https://www1.nseindia.com/content/indices/ind_nifty500list.csv)

In [2]:
ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
tickers_file = pd.read_csv(ticker_url)
tickers_df = tickers_file[['Symbol', 'Company Name']]
tickers = tickers_df['Symbol']
tickers.head()

0      3MINDIA
1          ABB
2          ACC
3       AIAENG
4    APLAPOLLO
Name: Symbol, dtype: object

In [3]:
tickers.shape

(501,)

Scrape Article Headlines and Dates

In [4]:
news_url = 'https://ticker.finology.in/company/'

test_url = 'https://ticker.finology.in/company/XYZ'
req = Request(url= test_url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
response = urlopen(req)
html = BeautifulSoup(response, 'lxml')
news_links = html.select('#newsarticles > a')  

len(news_links)

list_test = []
for link in news_links:
    title = link.find('span', class_='h6').text
    #separate date and time from datetime object
    date_time_obj = datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
    art_date = date_time_obj.date().strftime('%Y/%m/%d')
    art_time = date_time_obj.time().strftime('%H:%M')
    list_test.append(['SBIN', title, art_date, art_time])

In [5]:
# list to store article data
data = []
unavailable_tickers = []
companies_len = len(tickers)
length = 10

for i in range(length):
    print(tickers[i])
    req = Request(url= '{}/{}'.format(news_url, tickers[i]),headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
    response = urlopen(req)
    html = BeautifulSoup(response, 'lxml')
    news_links = html.select('#newsarticles > a')     #select('.newslink')  
    if len(news_links) == 0:
        print('No news found for {}'.format(tickers[i]))
        unavailable_tickers.append(tickers[i])
        # delete all ticker from tickers and tickers_df together #todo
        continue

    for link in news_links:
        title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        data.append([tickers[i], title, art_date, art_time])  
    ###if (i != 0 and i%200 == 0):
    ####print('sleeping')
    ####time.sleep(30)
df = pd.DataFrame(data, columns=['Ticker', 'Headline', 'Date', 'Time'])

for ticker in unavailable_tickers:
    if ticker in tickers:
        tickers.drop(ticker, inplace=True)

tickers_df.drop(tickers_df[tickers_df['Symbol'].isin(unavailable_tickers)].index.values, inplace=True)



3MINDIA
ABB
ACC
AIAENG
APLAPOLLO
AUBANK
AARTIDRUGS
AAVAS
ABBOTINDIA
ADANIENT


C:\Users\singh\AppData\Local\Temp\ipykernel_8632\46446300.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_df.drop(tickers_df[tickers_df['Symbol'].isin(unavailable_tickers)].index.values, inplace=True)


In [6]:
print(unavailable_tickers)

[]


In [7]:
pprint(df)

       Ticker                                           Headline        Date  \
0     3MINDIA        3m India informs about postal ballot notice  2022/12/24   
1     3MINDIA      3M India informs about trading window closure  2022/12/23   
2     3MINDIA  3M India’s promoter to exit PFAS manufacturing...  2022/12/22   
3     3MINDIA   3M India informs about loss of share certificate  2022/12/19   
4     3MINDIA  3M India informs about issuance of duplicate s...  2022/11/21   
..        ...                                                ...         ...   
469  ADANIENT               Adani Enterprises - Quaterly Results  2022/08/04   
470  ADANIENT  Adani Enterprises inks MoU with Israel Innovat...  2022/08/04   
471  ADANIENT  Adani Enterprises' arm acquires 400MHz of spec...  2022/08/02   
472  ADANIENT  Adani Enterprises planning to raise funds thro...  2022/07/29   
473  ADANIENT  Adani Enterprises' arm deposits Rs 100 crore a...  2022/07/19   

      Time  
0    12:59  
1    15:38  


Sentiment Analysis

In [8]:
'''import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer'''

"import nltk\nnltk.downloader.download('vader_lexicon')\nfrom nltk.sentiment.vader import SentimentIntensityAnalyzer"

In [9]:
vader = SentimentIntensityAnalyzer()
scores = df['Headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)

In [10]:
scores_df.shape

(474, 4)

In [11]:
new_df = pd.merge(left=df, right=scores_df, on=df.index.values).drop(['key_0'], axis=1)

In [12]:
new_df

,Ticker,Headline,Date,Time,neg,neu,pos,compound
0,3MINDIA,3m India informs about postal ballot notice,2022/12/24,12:59,0.000,1.000,0.000,0.0000
1,3MINDIA,3M India informs about trading window closure,2022/12/23,15:38,0.000,1.000,0.000,0.0000
2,3MINDIA,3M India’s promoter to exit PFAS manufacturing...,2022/12/22,12:27,0.000,1.000,0.000,0.0000
3,3MINDIA,3M India informs about loss of share certificate,2022/12/19,17:00,0.219,0.571,0.210,-0.0258
4,3MINDIA,3M India informs about issuance of duplicate s...,2022/11/21,16:09,0.000,0.784,0.216,0.2960
...,...,...,...,...,...,...,...,...
469,ADANIENT,Adani Enterprises - Quaterly Results,2022/08/04,18:29,0.000,1.000,0.000,0.0000
470,ADANIENT,Adani Enterprises inks MoU with Israel Innovat...,2022/08/04,15:35,0.000,0.606,0.394,0.4404
471,ADANIENT,Adani Enterprises' arm acquires 400MHz of spec...,2022/08/02,18:00,0.000,1.000,0.000,0.0000
472,ADANIENT,Adani Enterprises planning to raise funds thro...,2022/07/29,10:09,0.000,1.000,0.000,0.0000


In [13]:
final_df = new_df.groupby('Ticker').mean()

C:\Users\singh\AppData\Local\Temp\ipykernel_8632\2542017131.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  final_df = new_df.groupby('Ticker').mean()


In [14]:
final_df.shape

(10, 4)

In [15]:
final_df

,neg,neu,pos,compound
Ticker,,,,
3MINDIA,0.008760,0.932560,0.058680,0.071982
AARTIDRUGS,0.023298,0.926255,0.050447,0.040130
AAVAS,0.000000,0.973400,0.026600,0.040926
ABB,0.018820,0.912140,0.069040,0.081176
ABBOTINDIA,0.008383,0.926872,0.064723,0.080911
ACC,0.015040,0.920320,0.064640,0.080172
ADANIENT,0.001920,0.958860,0.039220,0.058188
AIAENG,0.000000,0.973667,0.026333,0.043257
APLAPOLLO,0.000000,0.963780,0.036220,0.060572


Get Company Sector, industry data

In [16]:
sector = []
industry = []
mCap = []
for i in range(length):
    meta = nse_eq(tickers[i])
    print(tickers[i])
    try:
        sector.append(meta['industryInfo']['macro'])
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        sector.append(np.nan)
        industry.append(np.nan)
        mCap.append(np.nan)
        continue
    pprint('Sector: {}'.format(meta['industryInfo']['macro']))
    try:
        industry.append(meta['industryInfo']['sector'])
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        industry.append(np.nan)
        mCap.append(np.nan)
        continue
    pprint('Industry: {}'.format(meta['industryInfo']['sector']))
    try:
        mCap.append(round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2))
    except KeyError:
        print('{} is not available'.format(tickers[i]))
        mCap.append(np.nan)
    print('market cap is Rs {}'.format(round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2)))
    print('\n')


3MINDIA
'Sector: Diversified'
'Industry: Diversified'
market cap is Rs 254.92


ABB
'Sector: Industrials'
'Industry: Capital Goods'
market cap is Rs 618.85


ACC
'Sector: Commodities'
'Industry: Construction Materials'
market cap is Rs 444.12


AIAENG
'Sector: Industrials'
'Industry: Capital Goods'
market cap is Rs 227.72


APLAPOLLO
'Sector: Industrials'
'Industry: Capital Goods'
market cap is Rs 329.72


AUBANK
'Sector: Financial Services'
'Industry: Financial Services'
market cap is Rs 419.57


AARTIDRUGS
'Sector: Healthcare'
'Industry: Healthcare'
market cap is Rs 40.85


AAVAS
'Sector: Financial Services'
'Industry: Financial Services'
market cap is Rs 143.74


ABBOTINDIA
'Sector: Healthcare'
'Industry: Healthcare'
market cap is Rs 300.34


ADANIENT
'Sector: Commodities'
'Industry: Metals & Mining'
market cap is Rs 4100.24




In [17]:
final_df.shape

(10, 4)

In [18]:
industry

['Diversified',
 'Capital Goods',
 'Construction Materials',
 'Capital Goods',
 'Capital Goods',
 'Financial Services',
 'Healthcare',
 'Financial Services',
 'Healthcare',
 'Metals & Mining']

In [19]:

final_df['sector'] = sector
final_df['industry'] = industry
final_df['mCap (Billion)'] = mCap

In [20]:
final_df = final_df.reset_index()

In [21]:
final_df

,Ticker,neg,neu,pos,compound,sector,industry,mCap (Billion)
0,3MINDIA,0.008760,0.932560,0.058680,0.071982,Diversified,Diversified,254.92
1,AARTIDRUGS,0.023298,0.926255,0.050447,0.040130,Industrials,Capital Goods,618.85
2,AAVAS,0.000000,0.973400,0.026600,0.040926,Commodities,Construction Materials,444.12
3,ABB,0.018820,0.912140,0.069040,0.081176,Industrials,Capital Goods,227.72
4,ABBOTINDIA,0.008383,0.926872,0.064723,0.080911,Industrials,Capital Goods,329.72
5,ACC,0.015040,0.920320,0.064640,0.080172,Financial Services,Financial Services,419.57
6,ADANIENT,0.001920,0.958860,0.039220,0.058188,Healthcare,Healthcare,40.85
7,AIAENG,0.000000,0.973667,0.026333,0.043257,Financial Services,Financial Services,143.74
8,APLAPOLLO,0.000000,0.963780,0.036220,0.060572,Healthcare,Healthcare,300.34
9,AUBANK,0.002120,0.944120,0.053740,0.095700,Commodities,Metals & Mining,4100.24


In [22]:
final_df = pd.merge(final_df, tickers_df, left_on='Ticker', right_on='Symbol').drop('Symbol', axis=1)

In [23]:
final_df.columns = ['Symbol', 'Negative', 'Neutral', 'Positive', 'Sentiment Score', 'Sector', 'Industry', 'MCap (Billion)', 'Company Name']

In [24]:
final_df

,Symbol,Negative,Neutral,Positive,Sentiment Score,Sector,Industry,MCap (Billion),Company Name
0,3MINDIA,0.008760,0.932560,0.058680,0.071982,Diversified,Diversified,254.92,3M India Ltd.
1,AARTIDRUGS,0.023298,0.926255,0.050447,0.040130,Industrials,Capital Goods,618.85,Aarti Drugs Ltd.
2,AAVAS,0.000000,0.973400,0.026600,0.040926,Commodities,Construction Materials,444.12,Aavas Financiers Ltd.
3,ABB,0.018820,0.912140,0.069040,0.081176,Industrials,Capital Goods,227.72,ABB India Ltd.
4,ABBOTINDIA,0.008383,0.926872,0.064723,0.080911,Industrials,Capital Goods,329.72,Abbott India Ltd.
5,ACC,0.015040,0.920320,0.064640,0.080172,Financial Services,Financial Services,419.57,ACC Ltd.
6,ADANIENT,0.001920,0.958860,0.039220,0.058188,Healthcare,Healthcare,40.85,Adani Enterprises Ltd.
7,AIAENG,0.000000,0.973667,0.026333,0.043257,Financial Services,Financial Services,143.74,AIA Engineering Ltd.
8,APLAPOLLO,0.000000,0.963780,0.036220,0.060572,Healthcare,Healthcare,300.34,APL Apollo Tubes Ltd.
9,AUBANK,0.002120,0.944120,0.053740,0.095700,Commodities,Metals & Mining,4100.24,AU Small Finance Bank Ltd.


In [25]:
final_df.iloc[5,6] = np.nan

In [26]:
final_df = final_df.dropna()

In [27]:
final_df

,Symbol,Negative,Neutral,Positive,Sentiment Score,Sector,Industry,MCap (Billion),Company Name
0,3MINDIA,0.008760,0.932560,0.058680,0.071982,Diversified,Diversified,254.92,3M India Ltd.
1,AARTIDRUGS,0.023298,0.926255,0.050447,0.040130,Industrials,Capital Goods,618.85,Aarti Drugs Ltd.
2,AAVAS,0.000000,0.973400,0.026600,0.040926,Commodities,Construction Materials,444.12,Aavas Financiers Ltd.
3,ABB,0.018820,0.912140,0.069040,0.081176,Industrials,Capital Goods,227.72,ABB India Ltd.
4,ABBOTINDIA,0.008383,0.926872,0.064723,0.080911,Industrials,Capital Goods,329.72,Abbott India Ltd.
6,ADANIENT,0.001920,0.958860,0.039220,0.058188,Healthcare,Healthcare,40.85,Adani Enterprises Ltd.
7,AIAENG,0.000000,0.973667,0.026333,0.043257,Financial Services,Financial Services,143.74,AIA Engineering Ltd.
8,APLAPOLLO,0.000000,0.963780,0.036220,0.060572,Healthcare,Healthcare,300.34,APL Apollo Tubes Ltd.
9,AUBANK,0.002120,0.944120,0.053740,0.095700,Commodities,Metals & Mining,4100.24,AU Small Finance Bank Ltd.


Plotting

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly
import plotly.express as px

In [29]:
fig = px.treemap(
    final_df, path=[px.Constant('Nifty 500'), 'Sector', 'Industry', 'Symbol'], values='MCap (Billion)', color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)
fig.show()

In [ ]:
'''
req = Request(url= '{}/{}'.format(news_url, tickers[0]),headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'})
response = urlopen(req)
html = BeautifulSoup(response)
data = [] 
news_links = html.select('.newslink')
for link in news_links:
    title = link.find('span', class_='h6').text
    date = link.find('small').text
    data.append([title, date])

df = pd.DataFrame(data, columns=['Title', 'Date'])

print(df)
'''

References

https://blog.devgenius.io/best-way-to-speed-up-a-bulk-of-http-requests-in-python-4ec75badabed